In [1]:
%load_ext autoreload
%autoreload 2

### giant otter vocalizations
- ~500 vocalizations from https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0112562

In [2]:
from joblib import Parallel, delayed
from tqdm.autonotebook import tqdm
import pandas as pd
import librosa
from datetime import datetime
import json

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from avgn.utils.paths import DATA_DIR, ensure_dir

In [5]:
DSLOC = DATA_DIR /"raw/otter/"

In [6]:
wavfiles = list(DSLOC.glob('zip_contents/*.wav'))
len(wavfiles)

453

In [7]:
wf_df = pd.DataFrame(columns = ['wav_loc', 'id', 'call_type'])
for wf in wavfiles:
    call_id, call_type = wf.stem[6:].split('_')
    wf_df.loc[len(wf_df)] = [wf, call_id, call_type]

In [8]:
wf_df[:3]

,wav_loc,id,call_type
0,/mnt/cube/tsainbur/Projects/github_repos/avgn_...,S357,snort
1,/mnt/cube/tsainbur/Projects/github_repos/avgn_...,S054,begging scream
2,/mnt/cube/tsainbur/Projects/github_repos/avgn_...,S121,contact call


In [9]:
len(wf_df)

453

In [10]:
wf_df.call_type.unique()

array(['snort', 'begging scream', 'contact call', 'bark', 'hah',
       'begging scream gradation', 'whistle', 'Ascending scream', 'growl',
       'whistle double', 'isolation call', 'whine', 'wavering scream',
       'close call', 'contact call gradation', 'hum', 'begging call',
       'hum short', 'hum gradation', 'suckling call', 'underwater call',
       'Distress call 1', 'begging call like', 'contact call like',
       'high whistle', 'low whistle', 'Distress call 2', 'hum like',
       'bark like', 'hum gradation like'], dtype=object)

In [11]:
len(wf_df.call_type.unique())

30

### create json

In [12]:
from avgn.utils.audio import get_samplerate
from avgn.utils.json import NoIndent, NoIndentEncoder
from datetime import datetime
import librosa
import json

In [13]:
DATASET_ID = "giant_otter"

In [14]:
# create a unique datetime identifier for the files output by this notebook
DT_ID = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
DT_ID

'2019-09-30_21-39-36'

In [16]:
for idx, row in tqdm(wf_df.iterrows(), total=len(wf_df)):
    
    json_out = (
        DATA_DIR
        / "processed"
        / DATASET_ID
        / DT_ID
        / "JSON"
        / (row.wav_loc.stem.replace(" ", "_")  + ".JSON")
    )
    
    rate = get_samplerate(row.wav_loc.as_posix())
    bout_duration = librosa.get_duration(filename = row.wav_loc.as_posix())
    
    # wav general information
    json_dict = {}
    json_dict['fid'] = row['id']
    json_dict["species"] = "Pteronura brasiliensis"
    json_dict["common_name"] = "Giant otter"
    json_dict["samplerate_hz"] = rate
    json_dict["original_wav"] = row.wav_loc.as_posix()
    json_dict["length_s"] = bout_duration
    json_dict["wav_loc"] = row.wav_loc.as_posix()
    json_dict["indvs"] = {
        "UNK": {
            'syllables':
            {
                "start_times": [0.0],
                "end_times": [bout_duration],
                "labels": [row.call_type]
            }
        }
    }
    json_txt = json.dumps(json_dict, cls=NoIndentEncoder, indent=2)
    
    # save json
    ensure_dir(json_out.as_posix())
    print(json_txt, file=open(json_out.as_posix(), "w"))
    